# Advanced Querying Mongo

Importing libraries and setting up connection

In [4]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database("companies")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [17]:
# Your Code
babelgum = list(db.companies.find({"name":"Babelgum"}))
print(f"Number of matches = {len(babelgum)}")
babelgum[0]['name']


Number of matches = 1


'Babelgum'

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [38]:
# Your Code
filter_q = {"number_of_employees":{"$gt":5000}  }   
sort_q =   [("number_of_employees",1)]


em = list(db.companies.find(filter_q).sort(sort_q).limit(20))

# 20 primeras empresas con más de 5000 empleados ordenadas
[em[i]['name'] for i in range(len(em))]

['Nintendo',
 'Hexaware Technologies',
 'Facebook',
 'OpenText',
 'CPM Braxis',
 'LSI',
 'Microchip Technologies',
 'Mediaset',
 'Mindray Medical International',
 'Dentsu',
 'Atmel',
 'Tata Communications',
 'Baidu',
 'Acxiom',
 'Acxiom',
 'SRA International',
 "Moody's",
 'ManTech',
 'Adobe Systems',
 'Broadcom']

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [40]:
filter_q = {"founded_year":{"$gte":2000, "$lte":2005}  
           }   

project_q = {"name":1,
             "founded_year" :1
            }


res = list(db.companies.find(filter_q,project_q))
# el len coincide con compass
print(len(res))
# 10 primeras
[res[i] for i in range(10)]

3734


[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_year': 2004},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'founded_year': 2004},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d95'),
  'name': 'StumbleUpon',
  'founded_year': 2002},
 {'_id': ObjectId('52cdef7c4bab8bd675297d96'),
  'name': 'Gizmoz',
  'founded_year': 2003},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9a'),
  'name': 'Helio',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297da1'),
  'name': 'Plaxo',
  'founded_year': 2002},
 {'_id': ObjectId('52cdef7c4bab8bd675297da5'),
  'name': 'Technorati',
  'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [43]:
filter_q = { "ipo.valuation_amount":{"$gt":100000000}, "founded_year": {"$lt":2010}   } 

project_q = {"name":1, "ipo":1}


res = list(db.companies.find(filter_q,project_q))

# el len coincide con compass
print(len(res))

# 3 primeras
[res[i] for i in range(3)]

42


[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [ ]:
# Your Code

### 6. All the companies that don't include the `partners` field.

In [ ]:
# Your Code

### 7. All the companies that have a null type of value on the `category_code` field.

In [ ]:
# Your Code

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [ ]:
# Your Code

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
# Your Code

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code